# Perform differential expression analysis between RA patients

**Author: Caroline Larkin**

**Research Group: Gottschalk Lab, Department of Immunology, University of Pittsburgh**

Notebook based on [link](https://liana-py.readthedocs.io/en/latest/notebooks/targeted.html)

In [1]:
import numpy as np
import pandas as pd
import anndata as ad 
import scanpy as sc

import celltypist
from celltypist import models

import liana as li

# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

import matplotlib.pyplot as plt
import seaborn as sns 

sample_ids = ["SA220", "SA222", "SA227"]
sample_labels = ["Naive", "Resistant", "Resistant"] 

data_dir = "./EMATB8322/"

# Download base models from celltypist
models.download_models()

/ihome/rgottschalk/cil8/envs/rnaseq/lib/python3.13/site-packages/celltypist/classifier.py:11: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  from scanpy import __version__ as scv
📂 Storing models in /ihome/rgottschalk/cil8/.celltypist/data/models
⏩ Skipping [1/60]: Immune_All_Low.pkl (file exists)
⏩ Skipping [2/60]: Immune_All_High.pkl (file exists)
⏩ Skipping [3/60]: Adult_COVID19_PBMC.pkl (file exists)
⏩ Skipping [4/60]: Adult_CynomolgusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [5/60]: Adult_Human_MTG.pkl (file exists)
⏩ Skipping [6/60]: Adult_Human_PancreaticIslet.pkl (file exists)
⏩ Skipping [7/60]: Adult_Human_PrefrontalCortex.pkl (file exists)
⏩ Skipping [8/60]: Adult_Human_Skin.pkl (file exists)
⏩ Skipping [9/60]: Adult_Human_Vascular.pkl (file exists)
⏩ Skipping [10/60]: Adult_Mouse_Gut.pkl (file exists)
⏩ Skipping [11/60]: Adult_Mouse_OlfactoryBulb.pkl (file exists)
⏩ Skipping [12/60]: Adult_Pig_Hippocampus.pkl (file exi

## 1. Load data

In [2]:
# Load celltypist immune model
model = models.Model.load(model = "Immune_All_High.pkl")

In [3]:
ads = []
for id, label in zip(sample_ids, sample_labels):
    cur_ad = sc.read_mtx(filename=f"{data_dir}{id}/processed_counts.mtx").transpose()
    gene_names = pd.read_csv(f"{data_dir}{id}/processed_gene_names.csv", header=None)
    cur_ad.var_names = gene_names.values.flatten()
    print()#print(gene_names.values.flatten())
    barcodes = pd.read_csv(f"{data_dir}{id}/processed_metadata.csv", usecols=["barcode"])
    cur_ad.obs_names = barcodes.values.flatten()
    cur_ad.obs_names_make_unique()
    cur_ad.obs["Group"] = [label]* len(barcodes)
    cur_ad.obs["Sample"] = [id] * len(barcodes)
    display(cur_ad)

    # create a temporary copy of adata to preprocess
    temp = cur_ad.copy()
    # Normalizing to median total counts
    sc.pp.normalize_total(temp, target_sum=10000)
    # Logarithmize the data
    sc.pp.log1p(temp)

    predictions = celltypist.annotate(temp, model=model)
    new_ad = predictions.to_adata();
    cur_ad.obs["cell_type"] = new_ad.obs["predicted_labels"]
    cell_counts = cur_ad.obs["cell_type"].value_counts()
    # removing cell types with low counts
    #print(cell_counts)
    keep_cell_types = cell_counts[cell_counts.values > 10]
    #print(keep_cell_types.index.values)
    cur_ad = cur_ad[cur_ad.obs["cell_type"].isin(keep_cell_types.index.values)]
    display(cur_ad)
    #break
    ads.append(cur_ad); 
    del temp
    del new_ad


AnnData object with n_obs × n_vars = 1832 × 16171
    obs: 'Group', 'Sample'

🔬 Input data has 1832 cells and 16171 genes
🔗 Matching reference genes in the model
🧬 4688 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


View of AnnData object with n_obs × n_vars = 1822 × 16171
    obs: 'Group', 'Sample', 'cell_type'

AnnData object with n_obs × n_vars = 5189 × 17673
    obs: 'Group', 'Sample'

🔬 Input data has 5189 cells and 17673 genes
🔗 Matching reference genes in the model
🧬 5000 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


View of AnnData object with n_obs × n_vars = 5173 × 17673
    obs: 'Group', 'Sample', 'cell_type'

AnnData object with n_obs × n_vars = 4774 × 16896
    obs: 'Group', 'Sample'

🔬 Input data has 4774 cells and 16896 genes
🔗 Matching reference genes in the model
🧬 4930 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


View of AnnData object with n_obs × n_vars = 4757 × 16896
    obs: 'Group', 'Sample', 'cell_type'

In [4]:
adata = sc.concat(ads, merge="same", join="inner")
adata.obs_names_make_unique()
#adata.X.dtype = np.int64
adata.X = adata.X.toarray()
adata.X[np.isnan(adata.X)] = 0 
display(adata)

AnnData object with n_obs × n_vars = 11752 × 15464
    obs: 'Group', 'Sample', 'cell_type'

In [5]:
sets = [set(x.var_names) for x in ads]
union = list(sets[0].intersection(*sets[1:]))
print("The total number of genes included in these samples is: ", len(union))

The total number of genes included in these samples is:  15464


In [6]:
condition_key = "Group"
groupby = "cell_type"
dea_results = {}
quiet = True

for cell_group in adata.obs[groupby].unique():
    # Select cell profiles
    ctdata = adata[adata.obs[groupby] == cell_group].copy()
    print("\n\n\n\n\n-------------------------------------------------------------------------------------\n\nHERE", cell_group); 
    print("CELL TYPE: ", cell_group, ctdata.obs["Group"].unique())

    # Check to see if cell type is present in both Groups
    if(len(np.unique(ctdata.obs["Group"])) < 2):
        # remove it from the dataset and continue 
        adata = adata[adata.obs[groupby] != cell_group]
        continue
        
    # Build DESeq2 object
    # NOTE: this data is actually paired, so one could consider fitting the patient label as a confounder
    dds = DeseqDataSet(
    	adata=ctdata,
    	design_factors=condition_key,
    	ref_level=[condition_key, 'Naive'], # set control as reference
    	refit_cooks=True,
    )

    # Compute LFCs
    dds.deseq2()
    # Contrast between stim and ctrl
    stat_res = DeseqStats(dds, contrast=[condition_key, 'Resistant', 'Naive'])
    # Compute Wald test
    stat_res.summary()
    # Shrink LFCs
    stat_res.lfc_shrink(coeff='Group[T.Resistant]') # {condition_key}_cond_vs_ref
    #stat_res.lfc_shrink(coeff='condition_Resistant_vs_Naive') # {condition_key}_cond_vs_ref

    dea_results[cell_group] = stat_res.results_df;
    






-------------------------------------------------------------------------------------

HERE Fibroblasts
CELL TYPE:  Fibroblasts ['Naive' 'Resistant']


/scratch/slurm-7854814/ipykernel_385153/3328906579.py:20: DeprecationWarning: ref_level is deprecated and no longer has any effect. It will beremoved in a future release.
/scratch/slurm-7854814/ipykernel_385153/3328906579.py:20: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.57 seconds.

Fitting dispersions...
... done in 1.51 seconds.

Fitting dispersion trend curve...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 2.81 seconds.

Fitting LFCs...
... done in 1.87 seconds.

Calculating cook's distance...
... done in 1.41 seconds.

Replacing 20 outlier genes.

Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Running Wald tests...
... done in 0.82 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: Group Resistant vs Naive
                baseMean  log2FoldChange     lfcSE      stat        pvalue  \
FO538757.2      0.124881        0.709580  0.242665  2.924109  3.454438e-03   
AP006222.2      0.184819        0.307082  0.162347  1.891515  5.855569e-02   
RP4-669L17.10   0.004709        0.699487  2.025944  0.345265  7.298952e-01   
RP11-206L10.9   0.027324        0.687360  0.668524  1.028176  3.038672e-01   
FAM87B          0.010583        0.606791  1.052979  0.576261  5.644385e-01   
...                  ...             ...       ...       ...           ...   
MT-CYB         23.069859       -0.192027  0.020851 -9.209348  3.281106e-20   
AL592183.1      0.029974        0.683871  0.671623  1.018237  3.085654e-01   
AC007325.4      0.037172        0.548139  0.539205  1.016569  3.093583e-01   
AC004556.1      0.078518       -1.554060  0.292649 -5.310324  1.094303e-07   
AC240274.1      0.155812        0.336482  0.202221  1.663930  9.612640e-02   



...

... done in 0.75 seconds.

/scratch/slurm-7854814/ipykernel_385153/3328906579.py:20: DeprecationWarning: ref_level is deprecated and no longer has any effect. It will beremoved in a future release.
/scratch/slurm-7854814/ipykernel_385153/3328906579.py:20: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
Fitting size factors...
... done in 0.02 seconds.



Shrunk log2 fold change & Wald test p-value: Group[T.Resistant]
                baseMean  log2FoldChange     lfcSE      stat    pvalue  \
FO538757.2      0.043387        0.003310  1.230527  0.098600  0.921456   
AP006222.2      0.032893       -0.000852  0.548893  0.010116  0.991929   
RP4-669L17.10   0.000000             NaN       NaN       NaN       NaN   
RP11-206L10.9   0.021635        0.001189  1.337273  0.047925  0.961776   
FAM87B          0.000000             NaN       NaN       NaN       NaN   
...                  ...             ...       ...       ...       ...   
MT-CYB         16.822940        0.014423  0.297027  0.190757  0.848716   
AL592183.1      0.050585        0.002700  1.349245  0.077851  0.937946   
AC007325.4      0.022922        0.001167  1.267130  0.043950  0.964945   
AC004556.1      0.004783       -0.009877  0.248485 -0.015059  0.987985   
AC240274.1      0.059818        0.002101  1.646228  0.072564  0.942153   

                   padj  
FO538757.2          N

Fitting dispersions...
... done in 0.43 seconds.

Fitting dispersion trend curve...
... done in 0.09 seconds.

Fitting MAP dispersions...
... done in 0.60 seconds.

Fitting LFCs...
... done in 0.56 seconds.

Calculating cook's distance...
... done in 0.04 seconds.

Replacing 29 outlier genes.

Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Running Wald tests...
... done in 0.52 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: Group Resistant vs Naive
                baseMean  log2FoldChange     lfcSE      stat    pvalue  \
FO538757.2      0.028776        0.824975  5.029180  0.164038  0.869701   
AP006222.2      0.057320       -0.054644  2.528680 -0.021610  0.982759   
RP4-669L17.10   0.006075        0.754460  6.216707  0.121360  0.903406   
RP11-206L10.9   0.009659        0.522548  6.204435  0.084222  0.932880   
FAM87B          0.002557        0.498455  6.204518  0.080337  0.935969   
...                  ...             ...       ...       ...       ...   
MT-CYB         11.464372       -0.415449  0.278997 -1.489076  0.136467   
AL592183.1      0.038131        0.847963  5.033978  0.168448  0.866231   
AC007325.4      0.005944        0.520408  6.204442  0.083877  0.933154   
AC004556.1      0.011990        0.008937  6.185019  0.001445  0.998847   
AC240274.1      0.032120        0.575354  5.014122  0.114747  0.908646   

                   padj  
FO538757.2          Na

... done in 0.78 seconds.



In [7]:
# concat results across cell types
dea_df = pd.concat(dea_results)
dea_df = dea_df.reset_index().rename(columns={'level_0': groupby,'level_1':'index'}).set_index('index')
dea_df.head()

# Shrink LFCs
#stat_res.lfc_shrink(coeff='Group[T.Resistant]') # {condition_key}_cond_vs_ref
#stat_res.results_df

,cell_type,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
index,,,,,,,
FO538757.2,Fibroblasts,0.124881,0.566587,0.280376,2.924109,0.003454,0.009842
AP006222.2,Fibroblasts,0.184819,-0.129691,0.169100,1.891515,0.058556,0.110536
RP4-669L17.10,Fibroblasts,0.004709,-0.001118,0.875506,0.345265,0.729895,NaN
RP11-206L10.9,Fibroblasts,0.027324,0.046824,0.611874,1.028176,0.303867,NaN
FAM87B,Fibroblasts,0.010583,-0.224822,0.633389,0.576261,0.564439,NaN


In [8]:
# PyDeseq Seems to intrdoce NAs for some p-values
# NOTE: there sometimes some NaN being introduced, best to double check that, in this case it's only for a single gene, but it might be a problem.
len(dea_df[dea_df.isna().any(axis=1)])
dea_df.to_csv("deseq_results.csv")

In [9]:
#adata = adata[adata.obs[condition_key]=='Resistant'].copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [10]:
lr_res = li.multi.df_to_lr(adata,
                           dea_df=dea_df,
                           resource_name='consensus', # NOTE: uses HUMAN gene symbols!
                           expr_prop=0.1, # calculated for adata as passed - used to filter interactions
                           groupby="cell_type",
                           stat_keys=['stat', 'pvalue', 'padj'],
                           use_raw=False,
                           complex_col='stat', # NOTE: we use the Wald Stat to deal with complexes
                           verbose=True,
                           return_all_lrs=False,
                           )

Using resource `consensus`.
Using `.X`!
Converting to sparse csr matrix!
0.39 of entities in the resource are missing from the data.


In [11]:
lr_res = lr_res.sort_values("interaction_stat", ascending=False, key=abs)
# Select all of the interactions with macrophages are the target
lr_res = lr_res[lr_res["target"] == "Macrophages"]
lr_res.head(10)

,ligand,receptor,ligand_complex,receptor_complex,source,ligand_stat,ligand_pvalue,ligand_padj,ligand_expr,ligand_props,...,receptor_pvalue,receptor_padj,receptor_expr,receptor_props,interaction_stat,interaction_pvalue,interaction_padj,interaction_expr,interaction_props,interaction
4104,VIM,CD44,VIM,CD44,Endothelial cells,-34.265628,2.552207e-257,1.283528e-254,3.681040,0.999127,...,2.489514e-09,2.646970e-08,1.337293,0.762220,-20.113885,1.244757e-09,1.323485e-08,2.509166,0.880674,VIM^CD44
8424,VIM,CD44,VIM,CD44,Fibroblasts,-32.616964,2.357450e-233,9.790353e-231,3.606308,0.998240,...,2.489514e-09,2.646970e-08,1.337293,0.762220,-19.289554,1.244757e-09,1.323485e-08,2.471801,0.880230,VIM^CD44
8329,CXCL12,CXCR4,CXCL12,CXCR4,Fibroblasts,30.101009,4.700240e-199,1.229026e-196,2.238536,0.780211,...,3.563692e-11,4.462986e-10,0.610545,0.376968,18.361089,1.781846e-11,2.231493e-10,1.424540,0.578589,CXCL12^CXCR4
8291,APOE,SDC2,APOE,SDC2,Fibroblasts,33.165903,3.340814e-241,1.814319e-238,2.378408,0.738561,...,3.169624e-03,1.353437e-02,0.227950,0.146645,18.058347,1.584812e-03,6.767183e-03,1.303179,0.442603,APOE^SDC2
8443,APOE,ABCA1,APOE,ABCA1,Fibroblasts,33.165903,3.340814e-241,1.814319e-238,2.378408,0.738561,...,2.307151e-01,4.837241e-01,0.494338,0.356669,17.182211,1.153576e-01,2.418620e-01,1.436373,0.547615,APOE^ABCA1
8382,APOE,SORL1,APOE,SORL1,Fibroblasts,33.165903,3.340814e-241,1.814319e-238,2.378408,0.738561,...,5.311427e-01,7.890612e-01,0.288026,0.225352,16.896083,2.655714e-01,3.945306e-01,1.333217,0.481956,APOE^SORL1
8368,ANGPTL2,ITGB1,ANGPTL2,ITGA5_ITGB1,Fibroblasts,-32.630403,1.520047e-233,6.707206e-231,0.715059,0.482988,...,4.684623e-01,7.411197e-01,0.629548,0.463546,-16.677693,2.342312e-01,3.705599e-01,0.672304,0.473267,ANGPTL2^ITGA5_ITGB1
8327,CXCL12,ITGAV,CXCL12,ITGAV,Fibroblasts,30.101009,4.700240e-199,1.229026e-196,2.238536,0.780211,...,6.259711e-03,2.474708e-02,0.245828,0.198840,16.417433,3.129855e-03,1.237354e-02,1.242182,0.489526,CXCL12^ITGAV
8447,TIMP2,CD44,TIMP2,CD44,Fibroblasts,-26.735568,1.817332e-157,3.054849e-155,1.093515,0.704145,...,2.489514e-09,2.646970e-08,1.337293,0.762220,-16.348856,1.244757e-09,1.323485e-08,1.215404,0.733183,TIMP2^CD44
8369,ANGPTL2,TLR4,ANGPTL2,TLR4,Fibroblasts,-32.630403,1.520047e-233,6.707206e-231,0.715059,0.482988,...,9.549634e-01,9.848294e-01,0.195188,0.163629,-16.286964,4.774817e-01,4.924147e-01,0.455124,0.323308,ANGPTL2^TLR4


In [12]:
lr_res.to_csv("differential_expression_ligand_receptor_naive_v_resistant_target_macs.csv", index=False)

In [13]:
lr_res["target"].value_counts()

target
Macrophages    1826
Name: count, dtype: int64